In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
%matplotlib inline

In [3]:
import pandas as pd
import numpy as np

In [4]:
import matplotlib.pyplot as plt
import seaborn as sns
sns.set_context('paper')
sns.set_style('white')

In [5]:
df = pd.read_csv('bills/bills_kns_documentbill_all_20200513.csv.gz')
df.head()

,DocumentBillID,BillID,GroupTypeID,GroupTypeDesc,ApplicationID,ApplicationDesc,FilePath,LastUpdatedDate
0,75133,17689,15,קטע מדברי הכנסת,1,DOC,http://fs.knesset.gov.il//16/law/16_ls_ptk_736...,2010-06-02 21:14
1,75136,19331,15,קטע מדברי הכנסת,1,DOC,http://fs.knesset.gov.il//16/law/16_ls_ptk_736...,2010-06-02 21:14
2,75137,20045,15,קטע מדברי הכנסת,1,DOC,http://fs.knesset.gov.il//16/law/16_ls_ptk_736...,2010-06-02 21:14
3,75138,20081,15,קטע מדברי הכנסת,1,DOC,http://fs.knesset.gov.il//16/law/16_ls_ptk_736...,2010-06-02 21:14
4,75139,20240,15,קטע מדברי הכנסת,1,DOC,http://fs.knesset.gov.il//16/law/16_ls_ptk_736...,2010-06-02 21:14


In [ ]:
df.groupby('B')

In [170]:
df.tail()

,DocumentBillID,BillID,GroupTypeID,GroupTypeDesc,ApplicationID,ApplicationDesc,FilePath,LastUpdatedDate,local_file_path
74325,496881,2086524,1,הצעת חוק לדיון מוקדם,1,DOC,http://fs.knesset.gov.il/23/law/23_lst_570364....,2020-05-11 16:09,docs/23/law/23_lst_570364.docx
74326,496882,2086611,1,הצעת חוק לדיון מוקדם,1,DOC,http://fs.knesset.gov.il/23/law/23_lst_570365....,2020-05-11 16:09,docs/23/law/23_lst_570365.docx
74327,496883,2086786,1,הצעת חוק לדיון מוקדם,1,DOC,http://fs.knesset.gov.il/23/law/23_lst_570366....,2020-05-11 16:09,docs/23/law/23_lst_570366.docx
74328,496884,2085760,1,הצעת חוק לדיון מוקדם,1,DOC,http://fs.knesset.gov.il/23/law/23_lst_570367....,2020-05-11 16:09,docs/23/law/23_lst_570367.docx
74329,496885,2086012,1,הצעת חוק לדיון מוקדם,1,DOC,http://fs.knesset.gov.il/23/law/23_lst_570368....,2020-05-11 16:09,docs/23/law/23_lst_570368.docx


In [6]:
df.shape

(74330, 8)

In [94]:
df.FilePath.nunique()

74330

In [7]:
import os
import requests

In [8]:
from pathlib import Path

def fetch_url(entry):
    path, uri = entry
    if not os.path.exists(path):
        path = Path(path)
        path.parent.mkdir(parents=True, exist_ok=True)
        r = requests.get(uri, stream=True)
        if r.status_code == 200:
            with open(path, 'wb') as f:
                for chunk in r:
                    f.write(chunk)
    return path



In [75]:
from urllib.parse import urlparse

def get_file_path_from_url(url):
    return urlparse(url).path.strip('/')

folder = 'docs'
urls = [(os.path.join(folder, get_file_path_from_url(x)), x.replace('http://', 'https://')) for x in df.FilePath.tolist()]
urls[:3]

[('docs/16/law/16_ls_ptk_73614.doc',
  'https://fs.knesset.gov.il//16/law/16_ls_ptk_73614.doc'),
 ('docs/16/law/16_ls_ptk_73617.doc',
  'https://fs.knesset.gov.il//16/law/16_ls_ptk_73617.doc'),
 ('docs/16/law/16_ls_ptk_73618.doc',
  'https://fs.knesset.gov.il//16/law/16_ls_ptk_73618.doc')]

In [96]:
from multiprocessing.pool import ThreadPool
results = ThreadPool(8).imap_unordered(fetch_url, urls)


In [97]:
len(os.listdir(folder))

23

In [120]:
sum([len(files) for r, d, files in os.walk(folder)])

72952

In [125]:
df['local_file_path'] = [x if os.path.exists(x) else None for x, y in urls ]

df.head()

,DocumentBillID,BillID,GroupTypeID,GroupTypeDesc,ApplicationID,ApplicationDesc,FilePath,LastUpdatedDate,local_file_path
0,75133,17689,15,קטע מדברי הכנסת,1,DOC,http://fs.knesset.gov.il//16/law/16_ls_ptk_736...,2010-06-02 21:14,docs/16/law/16_ls_ptk_73614.doc
1,75136,19331,15,קטע מדברי הכנסת,1,DOC,http://fs.knesset.gov.il//16/law/16_ls_ptk_736...,2010-06-02 21:14,docs/16/law/16_ls_ptk_73617.doc
2,75137,20045,15,קטע מדברי הכנסת,1,DOC,http://fs.knesset.gov.il//16/law/16_ls_ptk_736...,2010-06-02 21:14,docs/16/law/16_ls_ptk_73618.doc
3,75138,20081,15,קטע מדברי הכנסת,1,DOC,http://fs.knesset.gov.il//16/law/16_ls_ptk_736...,2010-06-02 21:14,docs/16/law/16_ls_ptk_73619.doc
4,75139,20240,15,קטע מדברי הכנסת,1,DOC,http://fs.knesset.gov.il//16/law/16_ls_ptk_736...,2010-06-02 21:14,docs/16/law/16_ls_ptk_73620.doc


In [126]:
df[df.local_file_path.isna()]

,DocumentBillID,BillID,GroupTypeID,GroupTypeDesc,ApplicationID,ApplicationDesc,FilePath,LastUpdatedDate,local_file_path
17222,207969,150572,9,חוק - פרסום ברשומות,4,PDF,http://fs.knesset.gov.il//11/law/11_lsr_210428...,2012-07-02 18:32,None
17223,207969,150572,9,חוק - פרסום ברשומות,1,DOC,http://fs.knesset.gov.il//11/law/11_lsr_214400...,2012-07-02 18:32,None
34848,275469,150195,3,הצעת חוק לקריאה הראשונה - נוסח מתוקן,3,PPT,http://fs.knesset.gov.il//3/law/3_lf1_292805.PDF,2014-07-07 14:50,None
34849,275470,147131,3,הצעת חוק לקריאה הראשונה - נוסח מתוקן,3,PPT,http://fs.knesset.gov.il//9/law/9_lf1_292806.PDF,2014-07-07 14:50,None
34850,275471,148562,3,הצעת חוק לקריאה הראשונה - נוסח מתוקן,3,PPT,http://fs.knesset.gov.il//9/law/9_lf1_292807.PDF,2014-07-07 14:50,None
...,...,...,...,...,...,...,...,...,...
73553,494780,2087322,2,הצעת חוק לקריאה הראשונה,4,PDF,http://fs.knesset.gov.il/23/law/23_ls1_568263.pdf,2020-04-06 13:22,None
73565,494979,2087390,2,הצעת חוק לקריאה הראשונה,4,PDF,http://fs.knesset.gov.il/23/law/23_ls1_568462.pdf,2020-04-16 07:42,None
73567,495060,2087470,2,הצעת חוק לקריאה הראשונה,4,PDF,http://fs.knesset.gov.il/23/law/23_ls1_568543.pdf,2020-04-20 09:27,None
73857,495602,2087720,2,הצעת חוק לקריאה הראשונה,4,PDF,http://fs.knesset.gov.il/23/law/23_ls1_569085.pdf,2020-04-23 18:47,None


In [107]:
dfc = pd.read_csv('committees/committees_kns_documentcommitteesession_all_20200513.csv.gz')
dfc.head()

/home/nlp/danb/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3058: DtypeWarning: Columns (16) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,DocumentCommitteeSessionID,CommitteeSessionID,GroupTypeID,GroupTypeDesc,ApplicationID,ApplicationDesc,FilePath,LastUpdatedDate,download_filename,download_filesize,...,text_protocol_extension,text_parsed_filename,text_filesize,text_crc32c,text_error,parts_protocol_extension,parts_parsed_filename,parts_filesize,parts_crc32c,parts_error
0,71333,65782,23,פרוטוקול ועדה,1,DOC,http://fs.knesset.gov.il/16/Committees/16_ptv_...,2010-06-02 20:32,files/23/7/1/71333.DOC,190976,...,.doc,files/6/5/65782.txt,82752,tp3JqQ==,NaN,.doc,files/6/5/65782.csv,82170,gJyRYA==,NaN
1,71335,66045,23,פרוטוקול ועדה,1,DOC,http://fs.knesset.gov.il/16/Committees/16_ptv_...,2010-06-02 20:32,files/23/7/1/71335.DOC,244736,...,.doc,files/6/6/66045.txt,100331,G3Koqw==,NaN,.doc,files/6/6/66045.csv,99769,o8UQAw==,NaN
2,71336,66046,23,פרוטוקול ועדה,1,DOC,http://fs.knesset.gov.il/16/Committees/16_ptv_...,2010-06-02 20:32,files/23/7/1/71336.DOC,42496,...,.doc,files/6/6/66046.txt,7175,d0Picw==,NaN,.doc,files/6/6/66046.csv,7124,GSb/9g==,NaN
3,71337,66050,23,פרוטוקול ועדה,1,DOC,http://fs.knesset.gov.il/16/Committees/16_ptv_...,2010-06-02 20:32,files/23/7/1/71337.DOC,92672,...,.doc,files/6/6/66050.txt,36175,UvKzdw==,NaN,.doc,files/6/6/66050.csv,36036,aZCFng==,NaN
4,71340,66166,23,פרוטוקול ועדה,1,DOC,http://fs.knesset.gov.il/16/Committees/16_ptv_...,2010-06-02 20:32,files/23/7/1/71340.DOC,126976,...,.doc,files/6/6/66166.txt,57541,E1mhzg==,NaN,.doc,files/6/6/66166.csv,57280,CSROTQ==,NaN


In [108]:
folder = 'docs'
curls = [(os.path.join(folder, get_file_path_from_url(x)), x.replace('http://', 'https://')) for x in dfc.FilePath.tolist()]
curls[:3]

[('docs/16/Committees/16_ptv_71307.doc',
  'https://fs.knesset.gov.il/16/Committees/16_ptv_71307.doc'),
 ('docs/16/Committees/16_ptv_71308.doc',
  'https://fs.knesset.gov.il/16/Committees/16_ptv_71308.doc'),
 ('docs/16/Committees/16_ptv_71309.doc',
  'https://fs.knesset.gov.il/16/Committees/16_ptv_71309.doc')]

In [109]:
dfc['local_file_path'] = [x for x, y in curls]
dfc.head()

,DocumentCommitteeSessionID,CommitteeSessionID,GroupTypeID,GroupTypeDesc,ApplicationID,ApplicationDesc,FilePath,LastUpdatedDate,download_filename,download_filesize,...,text_parsed_filename,text_filesize,text_crc32c,text_error,parts_protocol_extension,parts_parsed_filename,parts_filesize,parts_crc32c,parts_error,local_file_path
0,71333,65782,23,פרוטוקול ועדה,1,DOC,http://fs.knesset.gov.il/16/Committees/16_ptv_...,2010-06-02 20:32,files/23/7/1/71333.DOC,190976,...,files/6/5/65782.txt,82752,tp3JqQ==,NaN,.doc,files/6/5/65782.csv,82170,gJyRYA==,NaN,docs/16/Committees/16_ptv_71307.doc
1,71335,66045,23,פרוטוקול ועדה,1,DOC,http://fs.knesset.gov.il/16/Committees/16_ptv_...,2010-06-02 20:32,files/23/7/1/71335.DOC,244736,...,files/6/6/66045.txt,100331,G3Koqw==,NaN,.doc,files/6/6/66045.csv,99769,o8UQAw==,NaN,docs/16/Committees/16_ptv_71308.doc
2,71336,66046,23,פרוטוקול ועדה,1,DOC,http://fs.knesset.gov.il/16/Committees/16_ptv_...,2010-06-02 20:32,files/23/7/1/71336.DOC,42496,...,files/6/6/66046.txt,7175,d0Picw==,NaN,.doc,files/6/6/66046.csv,7124,GSb/9g==,NaN,docs/16/Committees/16_ptv_71309.doc
3,71337,66050,23,פרוטוקול ועדה,1,DOC,http://fs.knesset.gov.il/16/Committees/16_ptv_...,2010-06-02 20:32,files/23/7/1/71337.DOC,92672,...,files/6/6/66050.txt,36175,UvKzdw==,NaN,.doc,files/6/6/66050.csv,36036,aZCFng==,NaN,docs/16/Committees/16_ptv_71310.doc
4,71340,66166,23,פרוטוקול ועדה,1,DOC,http://fs.knesset.gov.il/16/Committees/16_ptv_...,2010-06-02 20:32,files/23/7/1/71340.DOC,126976,...,files/6/6/66166.txt,57541,E1mhzg==,NaN,.doc,files/6/6/66166.csv,57280,CSROTQ==,NaN,docs/16/Committees/16_ptv_71311.doc


In [151]:
dfc.shape

(138400, 23)

In [110]:
from multiprocessing.pool import ThreadPool
cresults = ThreadPool(8).imap_unordered(fetch_url, curls)


In [117]:
len(os.listdir('docs'))

23

In [159]:
sum([len(files) for r, d, files in os.walk(folder)])

210247

In [160]:
dfc['local_file_path'] = [x if os.path.exists(x) else None for x, y in curls ]

dfc.head()

,DocumentCommitteeSessionID,CommitteeSessionID,GroupTypeID,GroupTypeDesc,ApplicationID,ApplicationDesc,FilePath,LastUpdatedDate,download_filename,download_filesize,...,text_parsed_filename,text_filesize,text_crc32c,text_error,parts_protocol_extension,parts_parsed_filename,parts_filesize,parts_crc32c,parts_error,local_file_path
0,71333,65782,23,פרוטוקול ועדה,1,DOC,http://fs.knesset.gov.il/16/Committees/16_ptv_...,2010-06-02 20:32,files/23/7/1/71333.DOC,190976,...,files/6/5/65782.txt,82752,tp3JqQ==,NaN,.doc,files/6/5/65782.csv,82170,gJyRYA==,NaN,docs/16/Committees/16_ptv_71307.doc
1,71335,66045,23,פרוטוקול ועדה,1,DOC,http://fs.knesset.gov.il/16/Committees/16_ptv_...,2010-06-02 20:32,files/23/7/1/71335.DOC,244736,...,files/6/6/66045.txt,100331,G3Koqw==,NaN,.doc,files/6/6/66045.csv,99769,o8UQAw==,NaN,docs/16/Committees/16_ptv_71308.doc
2,71336,66046,23,פרוטוקול ועדה,1,DOC,http://fs.knesset.gov.il/16/Committees/16_ptv_...,2010-06-02 20:32,files/23/7/1/71336.DOC,42496,...,files/6/6/66046.txt,7175,d0Picw==,NaN,.doc,files/6/6/66046.csv,7124,GSb/9g==,NaN,docs/16/Committees/16_ptv_71309.doc
3,71337,66050,23,פרוטוקול ועדה,1,DOC,http://fs.knesset.gov.il/16/Committees/16_ptv_...,2010-06-02 20:32,files/23/7/1/71337.DOC,92672,...,files/6/6/66050.txt,36175,UvKzdw==,NaN,.doc,files/6/6/66050.csv,36036,aZCFng==,NaN,docs/16/Committees/16_ptv_71310.doc
4,71340,66166,23,פרוטוקול ועדה,1,DOC,http://fs.knesset.gov.il/16/Committees/16_ptv_...,2010-06-02 20:32,files/23/7/1/71340.DOC,126976,...,files/6/6/66166.txt,57541,E1mhzg==,NaN,.doc,files/6/6/66166.csv,57280,CSROTQ==,NaN,docs/16/Committees/16_ptv_71311.doc


In [161]:
dfc[dfc.local_file_path.isna()].shape

(177, 23)

In [166]:
ap = pd.concat([dfc.local_file_path, df.local_file_path]).reset_index()['local_file_path'].dropna().drop_duplicates()
ap.nunique(), ap[ap.str.endswith('.docx')].nunique(), ap[ap.str.endswith('.doc')].nunique(), ap[ap.str.endswith('.pdf')].nunique()

(210243, 21342, 58953, 22180)

In [165]:
import docx2txt

In [178]:
def process_docx_to_file(path):
    out_path = '/'.join(['txt']+path.split('/')[1:])+'.txt'
    if not os.path.exists(out_path):
        text = docx2txt.process(path)
        Path(out_path).parent.mkdir(parents=True, exist_ok=True)

        with open(out_path, 'w') as of:
            of.write(text)
        
    return out_path

In [179]:
process_docx_to_file('docs/23/law/23_lst_570368.docx')

'txt/23/law/23_lst_570368.docx.txt'

In [180]:
texts = ThreadPool(20).imap_unordered(process_docx_to_file, ap[ap.str.endswith('.docx')].tolist())

In [183]:
sum([len(files) for r, d, files in os.walk('txt')])

21352

In [203]:
from tqdm import tqdm

In [204]:
n = 0
with tqdm(total=21352) as pbar:
    for r, d, files in os.walk('txt'):
        for file in files:
            with open(os.path.join(r, file)) as f:
                for line in f:
                    n += len(line.split())
            pbar.update(1)

n

21353it [00:27, 780.31it/s]                            


64215482